In [13]:
import xarray
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import cartopy
from cartopy.util import add_cyclic_point

In [14]:
#National Centers for Environmental Prediction
#Best National Blend of Models CONUS Grids Time Series
ncep_nbm = 'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NBM/CONUS/Best'
ndap = xarray.open_dataset(ncep_nbm)
ndap

<xarray.Dataset>
Dimensions:                                                                                  (
                                                                                              x: 2345,
                                                                                              y: 1597,
                                                                                              time: 255,
                                                                                              time1: 242,
                                                                                              time2: 201,
                                                                                              ...
                                                                                              time14_bounds_1: 2,
                                                                                              time15_bounds_1: 2,
                                                                                              time16_bounds_1: 2,
                                                                                              time17_bounds_1: 2,
                                                                                              time18_bounds_1: 2,
                                                                                              time19_bounds_1: 2)
Coordinates: (12/46)
  * x                                                                                        (x) float32 ...
  * y                                                                                        (y) float32 ...
  * time                                                                                     (time) datetime64[ns] ...
    reftime                                                                                  (time) datetime64[ns] ...
  * time1                                                                                    (time1) datetime64[ns] ...
    reftime1                                                                                 (time1) datetime64[ns] ...
    ...                                                                                       ...
  * time19                                                                                   (time19) datetime64[ns] ...
    reftime19                                                                                (time19) datetime64[ns] ...
  * height_above_ground                                                                      (height_above_ground) float32 ...
  * height_above_ground1                                                                     (height_above_ground1) float32 ...
  * altitude_above_msl                                                                       (altitude_above_msl) float32 ...
  * height_above_ground2                                                                     (height_above_ground2) float32 ...
Dimensions without coordinates: time10_bounds_1, time11_bounds_1,
                                time12_bounds_1, time13_bounds_1,
                                time14_bounds_1, time15_bounds_1,
                                time16_bounds_1, time17_bounds_1,
                                time18_bounds_1, time19_bounds_1
Data variables: (12/52)
    LambertConformal_Projection                                                              int32 ...
    time10_bounds                                                                            (time10, time10_bounds_1) datetime64[ns] ...
    time11_bounds                                                                            (time11, time11_bounds_1) datetime64[ns] ...
    time12_bounds                                                                            (time12, time12_bounds_1) datetime64[ns] ...
    time13_bounds                                                                            (time13, time13_bounds_1) datetime64[ns] ...
    time1

In [15]:
Data_variable = 'Temperature_height_above_ground'
dvndap = ndap[Data_variable]
dvndap

<xarray.DataArray 'Temperature_height_above_ground' (time7: 267,
                                                     height_above_ground1: 1,
                                                     y: 1597, x: 2345)>
[999905655 values with dtype=float32]
Coordinates:
  * x                     (x) float32 -3.271e+03 -3.269e+03 ... 2.682e+03
  * y                     (y) float32 -263.8 -261.3 ... 3.787e+03 3.79e+03
  * time7                 (time7) datetime64[ns] 2024-09-12T01:00:00 ... 2024...
    reftime7              (time7) datetime64[ns] ...
  * height_above_ground1  (height_above_ground1) float32 2.0
Attributes: (12/13)
    long_name:                       Temperature @ Specified height level abo...
    units:                           K
    abbreviation:                    TMP
    grid_mapping:                    LambertConformal_Projection
    Grib_Variable_Id:                VAR_7-14--1-0_L103
    Grib2_Parameter:                 [0 0 0]
    ...                              ...
    Grib2_Parameter_Category:        Temperature
    Grib2_Parameter_Name:            Temperature
    Grib2_Level_Type:                103
    Grib2_Level_Desc:                Specified height level above ground
    Grib2_Generating_Process_Type:   Forecast
    Grib2_Statistical_Process_Type:  UnknownStatType--1

In [16]:
grid_ndap = ndap[dvndap.grid_mapping]
grid_ndap

<xarray.DataArray 'LambertConformal_Projection' ()>
[1 values with dtype=int32]
Attributes:
    grid_mapping_name:              lambert_conformal_conic
    latitude_of_projection_origin:  25.0
    longitude_of_central_meridian:  265.0
    standard_parallel:              25.0
    earth_radius:                   6371200.0
    _CoordinateTransformType:       Projection
    _CoordinateAxisTypes:           GeoX GeoY

In [17]:
lon_cm = grid_ndap.longitude_of_central_meridian
lat_po = grid_ndap.latitude_of_projection_origin
lat_sp = grid_ndap.standard_parallel
earth_radius = grid_ndap.earth_radius